# From unstructered data to it Provenance ni


<div style="text-align: right"> 
<h3>EScience - PPG IC/UFF 2017.1</h3>

<h4>Daniel Junior, Kid Valeriano</h4>

</div>


## Relembrando...
O projeto tem como base o artigo <strong>Provenance as data mining: Combining file system metadata with content analysis</strong>.
</hr>
<li>A ideia é utilizar técnicas de Mineração de Dados para reconstruir a proveniência de dados não-estruturados.</li>
<li>Os dados não-estruturados utilizados como dados de entrada para o projeto são arquivos textos contendo versões de um artigo até que o mesmo chegasse a sua edição final.</li>

## Arquitetura
<div width="100%" height="100%">
    <img src="images/arquitetura.png" height="400" width="800" style="margin: 0 auto;"/>
</div>

## Projeto
<ul>
    <li>Linguagem Python</li>
    <li>Biblioteca Sci-kit Learn</li>
    <li>Os dados usados no experimento são do repositório <strong><a href="https://l.facebook.com/l.php?u=http%3A%2F%2F137.207.234.78%2Fsearch%2Fdetail%2F&h=ATMRd4R-lyI7h1CWdhxvn6UWFuIQ6Sk5lh5mt_Yl5P4cZtgMLg2Eab3JBX_fKeG0-K69PJBwnMqh8BrtHvD__C71t0gmJg-NTkGuLNtxIehOx2CwfwQx_44sEEALKLVSY6y_fg">CiteSeerX</a></strong></li>
</ul>

## Projeto

Os passos a serem seguidos são:

<ol>
    <li>Dado um diretório que contenha as revisões de arquivos, devo ser capaz de recuperar o conteúdo de cada arquivo.</li>
    <li>Representar o conteúdo do arquivo de uma forma que seja possível aplicar algoritmos de Clustering.</li>
    <li>Executar o algoritmo para criar os clusters.</li>
    <li>Para cada arquivo em um cluster, recuperar os metados relativos ao dados de modificação do arquivo.</li>
    <li>Com os metadados em mãos, exportar para o PROV-Model, fazendo o armazenamento em um banco relacional (SQLite).</li>
    <li>Ser capaz de receber um arquivo de entrada e retornar uma representação gŕafica da proveniência do mesmo.</li>
</ol>


In [1]:
#imports
import os, time
from stat import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

CLUSTERS_NUMBER = 2

### Passo 1: Carregar os dados

In [2]:
PATH = "data"
filenames = []
docs = []
for path, dirs, files in os.walk(PATH):
    for filename in files:
        fullpath = os.path.join(path, filename)
        with open(fullpath, 'r') as f:
            data = f.read()
            filenames.append(fullpath)
            docs.append(data)        

print(filenames[:10])

['data\\d1\\doc427.txt', 'data\\d1\\doc428.txt', 'data\\d1\\doc429.txt', 'data\\d1\\doc430.txt', 'data\\d1\\doc431.txt', 'data\\d1\\doc432.txt', 'data\\d1\\doc433.txt', 'data\\d1\\doc434.txt', 'data\\d1\\doc435.txt', 'data\\d1\\doc436.txt']


In [3]:
train_data = shuffle(docs, random_state=42)[int(len(docs)*0.2):]
test_data = shuffle(docs, random_state=42)[:int(len(docs)*0.2)]

train_filenames = shuffle(filenames, random_state=42)[int(len(docs)*0.2):]
test_filenames = shuffle(filenames, random_state=42)[:int(len(docs)*0.2)]


### Passo 2: Representação vetorial dos dados

<div width="100%" height="100%">
    <img src="images/tfidf.png" style="margin: 0 auto;"/>
</div>

### Passo 2: Representação vetorial dos dados

In [4]:
tfidf_vectorizer = TfidfVectorizer(stop_words="english", max_features=200000,)
tfidf_matrix = tfidf_vectorizer.fit_transform(train_data)
print("Dimensões da matriz:")
print(tfidf_matrix.todense().shape)

Dimensões da matriz:
(929, 19739)


### Passo 3: Executar algoritmo de clustering

In [5]:
km = KMeans(n_clusters=CLUSTERS_NUMBER, init='k-means++', max_iter=100, n_init=1,verbose=False)
km.fit(tfidf_matrix.todense())
print(km.labels_[:30])

[0 0 0 0 1 1 0 1 1 0 1 0 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0]


### Passo 4: Recuperar os metadados dos arquivos de cada cluster

In [6]:
clusters_predict = km.predict(tfidf_vectorizer.transform(test_data))
cluster_0_idx =  [i for i, x in enumerate(clusters_predict) if x == 0]
filenames_cluster_0 = [test_filenames[x] for x in cluster_0_idx ]

for filename in filenames_cluster_0[:3]:
    st = os.stat(filename)
    print("filename:", filename)
    print("file modified:", time.asctime(time.localtime(st[ST_CTIME])))
    print("file size:", st[ST_SIZE])
    

filename: data\d2\doc307.txt
file modified: Mon May 22 15:05:42 2017
file size: 1776
filename: data\d2\doc45.txt
file modified: Mon May 22 15:05:42 2017
file size: 1513
filename: data\d2\doc595.txt
file modified: Mon May 22 15:05:42 2017
file size: 1685


In [7]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

for i in range(2):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()
    
#km.predict(tfidf_vectorizer.transform(test_data.data).todense())

Cluster 0:

NameError: name 'terms' is not defined

In [89]:
from IPython.core.display import HTML
from urllib.request import urlopen
HTML(urlopen('https://raw.githubusercontent.com/lmarti/jupyter_custom/master/custom.include').read().decode('utf-8'))